<a href="https://colab.research.google.com/github/davidurpani/course-v3/blob/master/nbs/dl2/02_fully_connected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
cd /gdrive/My Drive/course-v3/nbs/dl2

/gdrive/My Drive/course-v3/nbs/dl2


In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=4960)

In [4]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [5]:
x_train,y_train,x_valid,y_valid = get_data()

In [6]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [7]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [8]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [9]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [10]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [11]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [12]:
# num hidden
nh = 50

[Tinker practice](https://course.fast.ai/videos/?lesson=8&t=5255)

In [13]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [14]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [15]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [16]:
def lin(x, w, b): return x@w + b

In [17]:
t = lin(x_valid, w1, b1)

In [18]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.0233), tensor(1.0584))

In [19]:
def relu(x): return x.clamp_min(0.)

In [20]:
t = relu(lin(x_valid, w1, b1))

In [21]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.4315), tensor(0.6164))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [22]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [24]:
w1.mean(),w1.std()

(tensor(-4.9815e-05), tensor(0.0503))

In [25]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.6554), tensor(0.8883))

In [26]:
#export
from torch.nn import init

In [27]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [28]:
init.kaiming_normal_??

In [29]:
w1.mean(),w1.std()

(tensor(-7.4424e-05), tensor(0.0504))

In [30]:
t.mean(),t.std()

(tensor(0.5958), tensor(0.8727))

In [31]:
w1.shape

torch.Size([784, 50])

In [32]:
import torch.nn

In [33]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [34]:
torch.nn.Linear.forward??

In [35]:
torch.nn.functional.linear??

In [36]:
torch.nn.Conv2d??

In [37]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [38]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [39]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0083), tensor(0.7884))

In [40]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [41]:
%timeit -n 10 _=model(x_valid)

10 loops, best of 3: 22.1 ms per loop


In [42]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

In [43]:
x_valid.shape

torch.Size([10000, 784])

In [44]:
w1.shape

torch.Size([784, 50])

In [45]:
b1.shape

torch.Size([50])

In [46]:
model(x_valid).shape

torch.Size([10000, 1])

### Loss function: MSE

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6372)

In [47]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [48]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [49]:
y_train,y_valid = y_train.float(),y_valid.float()

In [50]:
preds = model(x_train)

In [51]:
preds.shape

torch.Size([50000, 1])

In [52]:
mse(preds, y_train)

tensor(23.4539)

### Gradients and backward pass

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6493)

In [53]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [54]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [55]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [56]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [57]:
forward_and_backward(x_train, y_train)

In [58]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [59]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [60]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [61]:
loss = forward(xt2, y_train)

In [62]:
loss.backward()

In [63]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

In [64]:
import torch
# Creating the graph
x = torch.tensor(1.0, requires_grad = True)
z = x ** 3
z.backward() #Computes the gradient 
print(x.grad.data) #Prints '3' which is dz/dx 

tensor(3.)


## Refactor model

### Layers as classes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7112)

In [65]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [66]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [67]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [68]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [69]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [70]:
%time loss = model(x_train, y_train)

CPU times: user 113 ms, sys: 0 ns, total: 113 ms
Wall time: 113 ms


In [71]:
%time model.backward()

CPU times: user 3.17 s, sys: 3.99 s, total: 7.16 s
Wall time: 7.18 s


In [72]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [73]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [74]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [75]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [76]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [77]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [78]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [79]:
%time loss = model(x_train, y_train)

CPU times: user 118 ms, sys: 1.22 ms, total: 119 ms
Wall time: 121 ms


In [80]:
%time model.backward()

CPU times: user 220 ms, sys: 13.8 ms, total: 233 ms
Wall time: 235 ms


In [81]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7484)

In [82]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [83]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [84]:
%time loss = model(x_train, y_train)

CPU times: user 111 ms, sys: 697 µs, total: 112 ms
Wall time: 112 ms


In [85]:
%time model.backward()

CPU times: user 204 ms, sys: 27 µs, total: 204 ms
Wall time: 205 ms


In [86]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [87]:
#export
from torch import nn

In [88]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [89]:
model = Model(m, nh, 1)

In [90]:
%time loss = model(x_train, y_train)

CPU times: user 96.2 ms, sys: 0 ns, total: 96.2 ms
Wall time: 99.9 ms


In [91]:
%time loss.backward()

CPU times: user 79.7 ms, sys: 0 ns, total: 79.7 ms
Wall time: 79.9 ms


## Export

In [92]:
!./notebook2script.py 02_fully_connected.ipynb

/bin/bash: ./notebook2script.py: /usr/bin/env: bad interpreter: Permission denied
